# Web Scrapping

Para tener un esquema general, siempre haremos los mismos pasos:


1.   Importar la página mediante get o page_source
2.   Transformar la web para ser legible con BeautifulSoup
3.   Buscar el elemento que nos interesa mediante find all() utilizando la clase o el tipo
4.   Extraer el texto de la sopa de letras mediante un .text

Durante esta explicación se utilizarán métodos que fueron usados para un ecommerce y un exchange de cyptos respectivamente. En el primer caso nuestro objetivo es extraer el nombre del producto, su precio, imagen... Mientras que en el segundo caso buscaremos información sobre varias criptomonedas tras pulsar un botón



# Instalación de Paquetes

No tienes por que instalar todos, por ejemplo la priemra celda es obligatoria menos por  **image** que sirve para extraer imagenes de la web.

De la sergunda celda podemos prescindir de image, nltk  y nltk download

La tercera celda solo servirá para instalar selenium pero necesitará el BeautifuSoup y otros paquetes de arriba

In [ ]:
!pip3 install beautifulsoup4 --user  # Paquete para poder leer bien las paginas, en vez de una sopa de letras    
!pip3 install lxml --user            # Leector del lenguaje de las web   
!pip3 install html5lib --user        # Lenguaje html                 
!pip3 install image                  # Paquete para reconocer imagenes            

In [ ]:
import requests                    # Paquete para llamar a la pagina web, hay dos uno es este y el otro selenyum            
from bs4 import BeautifulSoup      # Importamos el paquete para leer la pagina                         
from PIL import Image              # Importamos el paquete para leer la imagen                      
from os.path  import basename                                
from collections import Counter                                
import pandas as pd                                
                                
import re                          # Paquete request      
import nltk                               
import image                       # Imagen  

nltk.download("all")

In [ ]:
!pip install selenium                                                      # Pip install for selenium                                                                               
!apt-get update                                                                 
!apt install chromium-chromedriver                                                  

from selenium import webdriver                                             # Webdriver extrae la información de la web    
from selenium.webdriver.support.ui import WebDriverWait                    # Rapidez del bot a la hora de buscar la página y clicar                             


chrome_options = webdriver.ChromeOptions()                                 # Preparación del driver                
chrome_options.add_argument('--headless')                                                  
chrome_options.add_argument('--no-sandbox')                                                  
chrome_options.add_argument('--disable-dev-shm-usage')                                                  
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)     # Driver ready                                            
wait = WebDriverWait(driver, 10)                                           # Velocidad del driver especificada

# Importar y Transformar la web

Tenemos dos paquetes para hacer esto:

*   Request
*   Selenyum

**Veamos Request primero**








In [ ]:
response= requests.get("DESIRED_URL") #Ruta de la pagina y obtención de información

In [ ]:
soup= BeautifulSoup(response.text, 'html.parser') #Transformamos la información a algo legible

**Ahora veamos selenyum**

A diferencia de  Request tendremos que primero decirle a nuestro driver qué página queremos y luego guardar sus resultados, para finalmente transformarlos

In [ ]:
driver.get("DESIRED_URL") #Este paso no era necesario con request, pero si con Selenyum

In [ ]:
response= driver.page_source             #Fijate como aquí usamos .page_source en vez de get, su función es la misma

In [ ]:
soup = BeautifulSoup(pagina_1, 'html.parser')   #Transformamos la información a algo legible

# Encontrar información

Para obtener información sobre la web debemos abrir la web en una de nuestras pestañas y utilizar la herrmaienta Inspeccionar elemento. Aquí encontraremos todo el código HTML de la página web, de aquí debemos extraer el path, clase o referencia a los elementos de la página que nos interesan.

Existen varios recursos para encontrar lo que buscamos en la web:


**Request**

In [ ]:
texto = soup.find_all("h3")                          #Encontrar la información que tenga un titulo H1,H2,H3,H4.H5.H6
texto=soup.find_all(text=True)                       #Encontrar todo el texto de la página         
tabla = soup.find_all('table', class_= 'inbox')      #Encontraremos las tablas que sean clase inbox

**Selenyum**

In [ ]:
soup.findAll(class_ = "a-section a-spacing-base")    #Encontar mediante la clase usando Selenyum

# Extraer el texto

**Selenyum**

In [ ]:
soup.findAll(class_ = "a-section a-spacing-base")    #Encontar mediante la clase usando Selenyum

**Request**

In [ ]:
for i in texto:                                      # Ya habremos obtenido todas las lineas de h3, con esta loop pasaremos por todo el codigo extrayendo solo el texto                                       
    print(i.text) 


print(tabla.text)                                    #Las tablas no van por lieneas así que podemos hacer un.text directamente para extraer todo el texto

**Selenyum**

In [ ]:
for elemento in resultado:
    titulo_producto = elemento.find(class_= "a-size-base-plus a-color-base a-text-normal").get_text()  #Utilizamos get.text en este caso
    imagen_producto = elemento.find("img")                                                             #Tambien podemos realizar estas busquedas con selenyum



--------------------------------------------------------------------------------------



# Sección avanzada: Listado ICO

Comentamos por comprobar cuantas tablas hay para luego extraer la que nos interesa, usando selenium

In [ ]:
#Vamos a ver por cuantas tables hay y que clases hay dentro de ellas, para así poder seleccionarlas
for table in soup.find_all('table'):
    print(table.get('class'))

In [ ]:
#Solo tenemos una table en este caso pero debemos elegirla de todas forma con el siguiente comando:
tables = soup.find_all('table')
table = soup.find('table', class_='sc-e19573c7-2 gJWiZE cmc-table ''')

Crearemos una loop para buscar la información ya acotada, porque nuestra variable table es solo una table de la página web, para luego extraerla en un DataFrame.

Debemos tener en cuenta el comienzo de la loop, las tablas en html tienen tr y td, que es lo mismo que table row y table data. Por tanto, vamos a iterar por cada table row para encontrar cada table data y extraer de ella sus columnas, que es donde está nuestra información

In [ ]:
#Vamos a iterar nuestra tabla para extraer todas las rows y filtrar su información para nuestras columnas, finalmente la añadimos al DF
indice = ['name','symbol', 'ico_price','stage', 'start_date', 'end_date', 'goal', 'launchpad'] #Si es verdad que nos costaría escribir todos los indices si fueran 1k PERO podemos intentar hacerlo automatico 'facilmente'
df_ico = pd.DataFrame(columns= indice)

for row in table.tbody.find_all('tr'):                    
  columnas = row.find_all('td')

  if(columnas != []):
        #.find("span").text                                               #Hay varios a poner, al final lo importante es saber qeu existe span, text y contents  
        name= columnas[0].span.text                                       #.span.text (https://stackoverflow.com/questions/21823229/finding-next-occurring-tag-and-its-enclosed-text-with-beautiful-soup)
        symbol = columnas[0].span.find_next_sibling('span').text          #Nos dará el siguiente texto entre el tag span
        ico_price= columnas[1].text.strip()
        stage= columnas[2].span.contents[0].strip('&0.')
        start_date=columnas[3].text.strip()
        end_date=columnas[4].text.strip()
        goal=columnas[5].text.strip()
        launchpad=columnas[6].text.strip()
        df_ico = df_ico.append({'name': name, 'symbol' : symbol, 'ico_price' : ico_price, 'stage' : stage, 'start_date' : start_date, 'end_date' : end_date, 'goal' : goal, 'launchpad' : launchpad}, ignore_index=True)


# Pulsar un boton y seguir extrayendo data

Se puede dar el caso en el que necesitemos pulsar un boton, aquí tenía que pasar de la página 1 a la 2, luego a la 3 y así sucesivamente. 

Para ello, debemos comunicarle al driver que volvemos a la página principal y asegurarnos que estamos en ella

Para interactuar con la web de esta manera siempre deberemos usar Selenyum



In [ ]:
driver.get('DESIRED_URL')

In [ ]:
print(driver.current_url)

Y ahora es cuando viene lo interesante, en mi caso tuve varios problemas pues cuando corría la loop me ponía que había un error cuando intentaba pulsar el boton. En este caso salía que había más de un elmento, incluso infnormaba de que era banner.
Entonces borre el cache y cookies de la página web para ver qué pasaba cuando selenyum abría la página. Ahí encontre el código HTML del cookie banner, entonces:


*   Cree un range con el número de páginas que me interesaba visitar, junto a una condición para que solo clicara en el banner en la primera página, ya que una vez cerrado no volvía  a salir en las páginas posteriores
*   Copie el path del banner, que es ***uno de los mejores metodos para obtener un elemento*** 
*   Copie y configure el path del boton de número de página que quería cambiar, que coincide con nuestro range.
*   Inserte la misma loop que la página principal



In [ ]:
first= 3
for i in range (first,13):
  time.sleep(20)
  if i == first:
    driver.find_element('xpath','//*[@id="cmc-cookie-policy-banner"]/div[2]').click()
    driver.find_element('xpath',f'//*[@id="__next"]/div/div[1]/div[2]/div/div/div[3]/p/div/ul/li[{i}]').click() # Añadiendo f' al comienzo de nuestra string nos permitirá añadir variables a nuestra string
  else:
    driver.find_element('xpath',f'//*[@id="__next"]/div/div[1]/div[2]/div/div/div[3]/p/div/ul/li[{i}]').click()
  print(driver.current_url)
  web_ico2 = driver.page_source
  soup2 = BeautifulSoup(web_ico2, 'html.parser')
  table2 = soup.find('table', class_='sc-e19573c7-2 gJWiZE cmc-table ''')
  for row in table2.tbody.find_all('tr'):
    columnas = row.find_all('td')
    if(columnas != []):
          #.find("span").text
          name= columnas[0].span.text                                    #.span.text (https://stackoverflow.com/questions/21823229/finding-next-occurring-tag-and-its-enclosed-text-with-beautiful-soup)
          symbol = columnas[0].span.find_next_sibling('span').text          #Nos dará el siguiente texto entre el tag span
          ico_price= columnas[1].text.strip()
          stage= columnas[2].span.contents[0].strip('&0.')
          start_date=columnas[3].text.strip()
          end_date=columnas[4].text.strip()
          goal=columnas[5].text.strip()
          launchpad=columnas[6].text.strip()
          df_ico = df_ico.append({'name': name, 'symbol' : symbol, 'ico_price' : ico_price, 'stage' : stage, 'start_date' : start_date, 'end_date' : end_date, 'goal' : goal, 'launchpad' : launchpad}, ignore_index=True)
          print(df_ico.tail(), '\n')
          print('next')


In [ ]:
import pickle

df_ico.to_pickle( 'route/nombre_deseado.pkl')  #Guardemos toda nuestra data para no tener que volver a hacer scraping y poder trabajar cone lla